# Mini Project

Authored by Regitze Sydendal, Nikolette Zoe Pedersen & Mie Jonasson

## initialisation

We start by importing our python packages that we need for the code to run

In [1]:
import networkx as nx
import numpy as np
import random

We define a function for opening the directed graphs from a filename. In the description of the exercise, we were told a DiGraph would be fine for this exercise, but have at the same time noted, that this type does not account for duplicate edges, and will only have these appear once in the dataset for the graph.

In [2]:
def open_graph_file(filename):
    with open(filename,'rb') as f:
        graph = nx.read_adjlist(f, create_using=nx.DiGraph())
    return graph

We load a specific file, and save it in a variable

In [3]:
graph_1 = open_graph_file('p2p-Gnutella08-mod.txt')

## Random Surfer

We want to choose a starting node at random, and assign it as 'current'

In [4]:
current = random.choice(list(graph_1))

We create a list of the dangling nodes in the graph

In [5]:
branching = np.array([len(list(graph_1.neighbors(n))) for n in graph_1])
dangling = [n for n,i in zip(list(graph_1),range(len(graph_1))) if branching[i]==0]

We initiate a dict for counting the number of times each node is visited in the random walk we are about to condone

In [34]:
visit_dict = {n:0 for n in list(graph_1)}

We make a loop that iterates a great number of times, and for each iteration it should travel to another node, and add a 'point' to the number of times that node has been visited

In [43]:
iterations = 1100000
visit_dict[current] += 1
for i in range(iterations):
    if random.random() < 0.15 or current in dangling:
        current = random.choice(list(graph_1))
    else:
        current = random.choice(list(graph_1.neighbors(current)))
    visit_dict[current] += 1

We now sort the keys of the dictionary to have the ones being visited most (having the highest value) at the beginning, thus decreasing in popularity towards the end, and then printing the 10 most popular nodes:

In [44]:
most_popular = sorted(visit_dict.items(), key = lambda kv: kv[1])
most_popular.reverse()
most_popular_nodes = [n for n,v in most_popular]
print(most_popular_nodes[:10])

['367', '249', '145', '264', '266', '123', '5', '127', '122', '1317']


## Page Rank

#### Initialising

We use this command to get the number of nodes in the graph

In [4]:
n_nodes = len(graph_1)

We find the reverse of the graph to help us compute the backlinks later on

In [5]:
rev_graph = graph_1.reverse()

We create an array called 'branching' that will consist of the number of nodes that are linked to from the node with the value of the index

In [6]:
branching = np.array([len(list(graph_1.neighbors(n))) for n in graph_1])

We create a list consisting of the indices of the dangling nodes in the graph, by finding out which entries in branching are equal to 0

In [7]:
dangling_i = [n for n in range(len(graph_1)) if branching[n]==0]

This is the inital approximation for the ranking vector, given by x0, where all entries correspond to 1/n

In [8]:
x0 = np.array([[1/n_nodes] for _ in range(n_nodes)])

#### Estimating rankings

We create the variable xk, and give it our initial ranking vector, give m a value of 0.15 and choose the number of iterations

In [55]:
xk = x0
m = 0.15
iterations = 6

We now do a loop that continuously computes a new xk (which will be xk+1) and assigns this to be the new xk

In [70]:
for j in range(iterations):
    xk1 = np.zeros((n_nodes,1))
    #Our Dxki will be the same for all i, so we compute it once, and add it on later.
    Dxki = sum([xk[ind]/n_nodes for ind in dangling_i])
    for i,n in zip(range(n_nodes),graph_1):
        #We initialise x(k+1)i as m*(Sxk)i, and since we know that every entry in Sxk is given by sum(xk)/n,
        #And we furthermore know that the sum(xk) i always 1, the value of m*(Sxk)i will always be m * 1/n
        xk1i = m * (1/n_nodes)
        
        #We compute Axki by looping over the backlinks, and adding the nodes own rank divided by 
        #the number of links going out from the node.
        Axki = sum([xk[ind]/branching[ind] for ind,n2 in zip(range(n_nodes),rev_graph) if n2 in rev_graph.neighbors(n)])
        
        #We add the different parts all together and place it back into xk+1
        xk1i += (1-m)*Dxki+(1-m)*Axki
        xk1[i]=xk1i
    #Before we begin a new iteration, we make xk take on the values of x(k+1)
    xk = xk1

We print the top 10 rankings, and check that the sum of them is 1.

In [71]:
xk_for_sort = np.concatenate((np.reshape(np.asarray(range(n_nodes)),(n_nodes,1)),xk),axis = 1)
xksorted = xk_for_sort[xk_for_sort[:,1].argsort()]
xksorted = np.flip(xksorted)
print('10 most popular nodes: \n',xksorted[:10,1])
print('Their Rankings: \n',xksorted[:10,0])
print('\nSum of the rankings: ',round(sum(xk)[0],4))

10 most popular nodes: 
 [ 82.  36.  41.  37. 613.  79.  32. 462. 452.   5.]
Their Rankings: 
 [0.00238765 0.0021843  0.00205489 0.00199875 0.00196344 0.00186349
 0.00186051 0.00185325 0.0018435  0.00183107]

Sum of the rankings:  1.0
